In [57]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [58]:
house_dict = []

for page in range(15, 21):
#     Filter Out Coming Soon, Auction, Home Type: Multi-Family, Days On Zillow: 90 Days
            
#     Agency Listing
    linked = 'https://www.zillow.com/ca/duplex/' + str(page) + '_p/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-132.16065190625%2C%22east%22%3A-106.45264409375%2C%22south%22%3A29.712790718202044%2C%22north%22%3A44.40704031881662%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A9%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22sf%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A6%2C%22category%22%3A%22cat1%22%2C%22pagination%22%3A%7B%22currentPage%22%3A' + str(page) + '%7D%7D'
    
#     Other Listing
#     linked = 'https://www.zillow.com/ca/duplex/' + str(page) + '_p/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-132.16065190625%2C%22east%22%3A-106.45264409375%2C%22south%22%3A29.712790718202044%2C%22north%22%3A44.40704031881662%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A9%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22sf%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A6%2C%22category%22%3A%22cat2%22%2C%22pagination%22%3A%7B%22currentPage%22%3A' + str(page) + '%7D%7D'

    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    print('page: ', page)
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            print(house_website)
            # Identify and return house data
#             listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
#             print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")
            lot_search = loop_soup.findAll('div', class_="sc-pJipy hgzjTs")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in lot_search:
#                 print(content.text)
                if "LotLot size" in content.text.split(":")[0]:
                    lot = content.text.split(": ")[1];
                    lot = lot.replace('Lot features', "")
                    
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
            print(lot)
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre") or (str(lot.split(" ")[1]) == "Acres") or (str(lot.split(" ")[1]) == "Acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
            if (len(loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")) < 3):
                zestimate = 'null'
                rent_zestimate = 'null'
            else:
                zestimate = loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")[0].text
                rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")[2].text
            if (zestimate[0] != '$'):
                zestimate = 'null';
            print('zestimate', zestimate)
            print('rent zestimate', rent_zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
#                 "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(5)
            
        except AttributeError as e:
            continue

# print(house_dict)

page:  15
https://www.zillow.com/homedetails/144-Clearbrook-19-Irvine-CA-92614/25752045_zpid/
null
zestimate $572,200
rent zestimate $2,400/mo
https://www.zillow.com/homedetails/28212-Sorrento-UNIT-90-Laguna-Niguel-CA-92677/25761249_zpid/
null
zestimate $576,300
rent zestimate $2,765/mo
https://www.zillow.com/homedetails/852-Lakeview-Ave-South-Lake-Tahoe-CA-96150/83901779_zpid/
8,276 sqft
zestimate $650,000
rent zestimate $3,869/mo
https://www.zillow.com/homedetails/2045-S-5th-St-Fresno-CA-93702/2110718807_zpid/
6,351 sqft
zestimate $304,545
rent zestimate $1,814/mo
https://www.zillow.com/homedetails/1269-Citrus-Ave-Chico-CA-95926/18214172_zpid/
9,583 sqft
zestimate $308,010
rent zestimate $2,011/mo
https://www.zillow.com/homedetails/475-Wyoming-St-Pasadena-CA-91103/20907787_zpid/
8,251 sqft
zestimate $610,500
rent zestimate $3,899/mo
https://www.zillow.com/homedetails/27-Crescent-Ave-Sausalito-CA-94965/2072146712_zpid/
6,651 sqft
zestimate null
rent zestimate null
https://www.zillow.c

In [59]:
house_data = pd.DataFrame(house_dict)
house_data

,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zestimate,Rent_zestimate,Zillow_price/sqft,House_website
0,"$549,900","144 Clearbrook #19, Irvine, CA 92614",144 Clearbrook #19,Irvine,CA,92614,2,2,970,null,null,MultiFamily,1980,1 Carport space,$432 monthly,"$572,200","$2,400/mo",$567,https://www.zillow.com/homedetails/144-Clearbr...
1,"$562,900","28212 Sorrento UNIT 90, Laguna Niguel, CA 92677",28212 Sorrento UNIT 90,LagunaNiguel,CA,92677,3,2,"1,171",null,null,MultiFamily,1984,1 Garage space,$452 monthly,"$576,300","$2,765/mo",$481,https://www.zillow.com/homedetails/28212-Sorre...
2,"$650,000","852 Lakeview Ave, South Lake Tahoe, CA 96150",852 Lakeview Ave,SouthLakeTahoe,CA,96150,9,6,"3,016","8,276 sqft",8276,Quadruplex,1939,4 Parking spaces,null,"$650,000","$3,869/mo",$216,https://www.zillow.com/homedetails/852-Lakevie...
3,"$300,000","2045 S 5th St, Fresno, CA 93702",2045 S 5th St,Fresno,CA,93702,4,2,"1,800","6,351 sqft",6351,MultiFamily,1960,2 Parking spaces,null,"$304,545","$1,814/mo",$167,https://www.zillow.com/homedetails/2045-S-5th-...
4,"$299,000","1269 Citrus Ave, Chico, CA 95926",1269 Citrus Ave,Chico,CA,95926,3,3,"1,710","9,583 sqft",9583,MultiFamily,1905,4 Parking spaces,null,"$308,010","$2,011/mo",$175,https://www.zillow.com/homedetails/1269-Citrus...
5,"$550,000","475 Wyoming St, Pasadena, CA 91103",475 Wyoming St,Pasadena,CA,91103,--,--,--,"8,251 sqft",8251,Multi Family,1910,Driveway,null,"$610,500","$3,899/mo",null,https://www.zillow.com/homedetails/475-Wyoming...
6,"$1,395,000","27 Crescent Ave, Sausalito, CA 94965",27 Crescent Ave,Sausalito,CA,94965,3,2,"1,552","6,651 sqft",6651,Duplex,1949,1 Open Parking space,null,null,null,$899,https://www.zillow.com/homedetails/27-Crescent...
7,"$495,000","3969 Becky Ct, Carmichael, CA 95608",3969 Becky Ct,Carmichael,CA,95608,4,2,"1,764",0.31 Acres,13503.6,Duplex,1963,2 Attached Garage spaces,null,"$504,400","$1,813/mo",$281,https://www.zillow.com/homedetails/3969-Becky-...
8,"$345,000","3546 Treleaven Ct, Antelope, CA 95843",3546 Treleaven Ct,Antelope,CA,95843,2,2,940,"4,077 sqft",4077,MultiFamily,1981,2 Covered Parking spaces,null,"$373,138","$1,550/mo",$367,https://www.zillow.com/homedetails/3546-Trelea...
9,"$549,900","144 Clearbrook #19, Irvine, CA 92614",144 Clearbrook #19,Irvine,CA,92614,2,2,970,null,null,MultiFamily,1980,1 Carport space,$432 monthly,"$572,200","$2,400/mo",$567,https://www.zillow.com/homedetails/144-Clearbr...


In [60]:
# Export file as a CSV, without the Pandas index, but with the header
# house_data.to_csv("Output/Zillow/Q2-2021_Multi_Family_Data_AgentListing_page15-20.csv", index=False, header=True)
house_data.to_csv("Output/Zillow/Q2-2021_Multi_Family_Data_OtherListing_page15-20.csv", index=False, header=True)